# Hymba+ 모델 비교 노트북

다양한 설정(Transformer 비율, MoE 여부 등)을 비교하여
파라미터 수 및 출력 분포를 확인합니다.

In [ ]:
import torch
import matplotlib.pyplot as plt

from hymba_plus.core.config import HymbaPlusConfig
from hymba_plus.models.language_model import HymbaPlus

plt.rcParams['font.family'] = ['NanumGothic', 'AppleGothic', 'Malgun Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def make_config(transformer_ratio: float):
    base = HymbaPlusConfig.from_yaml('../configs/hymba_plus.yaml')
    base.architecture.transformer_ratio = transformer_ratio
    return base

configs = {
    'Transformer': make_config(1.0),
    'Hybrid': make_config(0.5),
    'SSM': make_config(0.0),
}

In [ ]:
stats = {}
for name, cfg in configs.items():
    model = HymbaPlus(cfg)
    params = sum(p.numel() for p in model.parameters())
    input_ids = torch.randint(0, cfg.vocab_size, (2, 16))
    outputs = model(input_ids)
    stats[name] = {
        'params': params,
        'logits_mean': outputs.logits.mean().item(),
        'logits_std': outputs.logits.std().item(),
    }
stats

In [ ]:
labels = list(stats.keys())
values = [stats[k]['params'] for k in labels]

plt.figure(figsize=(6, 3))
plt.bar(labels, values)
plt.title('모델별 파라미터 수 비교')
plt.ylabel('파라미터 수')
plt.show()